# Cracking the Code: Predicting Puzzle Difficulty with Machine Learning

#### *How I Built an AI System to Solve the "Goldilocks Problem" in Game Design*

---

### The Hidden Architecture of Engagemen


Every great puzzle hides two stories: the one a player solves, and the one that decides what is worth solving. This project is about the second story, a system that doesn't just brute-force solutions, but studies what makes a board compelling, predicts how it will play, and curates puzzles that feel sharp, fair, and memorable

- **The core challenge**: How do you systematically identify the difference between a puzzle that teaches and one that frustrates? Between complexity that engages and complexity that overwhelms?

- **My approach**: Build a machine learning system that understands puzzle difficulty the way humans experience it — not just as mathematical complexity, but as cognitive load, spatial reasoning patterns, and the satisfying progression from confusion to clarity
---

### The Technical Challenge: Klotski Sliding Block Puzzles

I chose **Klotski puzzles** as my proving ground - these ancient Chinese sliding block puzzles are:
- **Computationally complex**: Even small changes create dramatically different difficulty
- **Measurable**: Clear success metrics (solvable/unsolvable, solution length)
- **Generalizable**: Principles apply to any spatial reasoning game

**The Goal**: Can I build a system that predicts puzzle difficulty more accurately than human intuition?

---

### What I Built: End-to-End ML Pipeline

**Custom Puzzle Solver**: BFS algorithm finding optimal solutions  
**Massive Dataset**: 50,000 unique puzzles with 22+ engineered features  
**Multi-Model ML Suite**: Classification, regression, clustering, and complexity analysis  
**Novel Feature Engineering**: Complexity scoring and symmetry detection algorithms  
**Business Intelligence**: Actionable insights for game design optimization  

---

### The Results That Matter

- **39.7% natural solvability rate** in randomly generated puzzles (perfect for balanced difficulty)
- **22+ engineered features** capturing spatial, compositional, and strategic complexity
- **Advanced complexity scoring** that correlates with human perception of difficulty
- **Symmetry detection algorithms** revealing hidden puzzle patterns
- **Production-ready pipeline** generating insights for game designers

---

### Why This Matters

This isn't just about puzzles. This is about **using data science to understand human cognition and engagement**. The techniques I've developed here apply to:

- **Educational software**: Adaptive difficulty in learning platforms and skill progression
- **Game design**: Any puzzle or strategy game balancing across player skill levels
- **User experience**: Predicting task complexity in any interface or workflow
- **Product onboarding**: Sequencing feature introduction to minimize cognitive overload

---

*Let's dive into the data and see how machine learning can solve the Goldilocks Problem...*


## 📋 Project Overview

### Technical Stack
- **Data Generation**: Custom Python pipeline with BFS solver
- **Feature Engineering**: 22+ domain-specific features
- **ML Framework**: scikit-learn, pandas, numpy
- **Visualization**: matplotlib, seaborn, plotly
- **Analysis**: Statistical modeling and pattern recognition

### Dataset Specifications
- **Size**: 50,000 unique Klotski puzzle boards generated
- **Features**: 22+ engineered features per puzzle
- **Target Variables**: Solvability (binary) + Solution Length (regression)
- **Quality**: Clean, validated data with proper feature encoding

In [ ]:
# Import all necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import warnings
from tabulate import tabulate
from collections import namedtuple
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report, confusion_matrix, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from IPython.display import SVG, display
import glob
import ast
import math
import os

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Plotly settings for  visuals
import plotly.io as pio
pio.templates.default = "simple_white"

print("Klotski Puzzle ML Analysis")
print("Libraries loaded successfully!")


Klotski Puzzle ML Analysis
Libraries loaded successfully!


## 📊 Data Loading and Initial Exploration

In [185]:
def load_dataset():
    """
    Load the dataset.
    """
    df = None  # Initialize df to avoid NameError


    possible_files = [
        '.\enhanced_dataset_20250912_024232.csv'
        ]
        
    for file_path in possible_files:
            if os.path.exists(file_path):
                if file_path.endswith('.json'):
                    df = pd.read_json(file_path)
                else:
                    df = pd.read_csv(file_path)
                print(f"📂 Loaded: {file_path}")
                break

    return df

df = load_dataset()  # Load the dataset

📂 Loaded: .\enhanced_dataset_20250912_024232.csv


In [186]:
print(f"\n📊 Dataset Overview:")
print(f"Shape: {df.shape[0]:,} puzzles × {df.shape[1]} features")

# Column Overview (data types)
print(f"\n🔍 Column Overview:")
print(df.dtypes.value_counts())



📊 Dataset Overview:
Shape: 50,000 puzzles × 24 features

🔍 Column Overview:
int64      15
object      5
float64     3
bool        1
Name: count, dtype: int64


In [187]:
# Set pandas display options to show more text
pd.set_option('display.max_colwidth', None)  # Show full content
pd.set_option('display.width', None)  # Don't wrap lines
pd.set_option('display.max_columns', None)  # Show all columns

# Custom display function for specific columns
sample_df = df.head().copy()
    
# Display basic info first
print(f"📋 First 5 puzzles:")
display(sample_df.drop(columns=['board_visual'], errors='ignore'))
    
# Then show board visuals separately
if 'board_visual' in df.columns:
    print("\n Board Visuals for these samples:")
    for i, (idx, row) in enumerate(sample_df.iterrows()):
        print(f"\n--- Puzzle {i+1} (Index {idx}) ---")
        print(row['board_visual'])

📋 First 5 puzzles:


,puzzle_id,timestamp,is_solvable,total_blocks,block_count_1x1,block_count_1x2,block_count_2x1,block_count_2x2,goal_initial_row,goal_initial_col,goal_distance_to_target,goal_manhattan_distance,blocks_between_goal_target,adjacent_1x1_count,adjacent_1x2_count,adjacent_2x1_count,wall_adjacent_sides,empty_spaces_count,empty_space,solution_length,solve_time_seconds,step_number,initial_block_states
0,puzzle_000000,2025-09-12T02:42:36.004653,True,11,6,2,2,1,1,2,2.236068,3,6,5,1,0,1,2,"[(3, 2), (4, 2)]",17,2.848788,NaN,"[(2, 2, 1, 2), (1, 2, 0, 1), (1, 1, 3, 3), (1, 1, 1, 1), (1, 2, 4, 0), (1, 1, 4, 3), (2, 1, 2, 0), (1, 1, 0, 3), (1, 1, 3, 1), (2, 1, 0, 0), (1, 1, 2, 1)]"
1,puzzle_000001,2025-09-12T02:42:33.172049,False,9,2,1,5,1,1,1,2.000000,2,2,2,1,5,0,2,"[(0, 3), (4, 0)]",0,0.014643,NaN,"[(2, 2, 1, 1), (2, 1, 1, 3), (1, 1, 3, 0), (2, 1, 3, 3), (1, 2, 0, 1), (2, 1, 0, 0), (2, 1, 3, 1), (2, 1, 3, 2), (1, 1, 2, 0)]"
2,puzzle_000002,2025-09-12T02:42:55.841823,True,10,4,2,3,1,0,2,3.162278,4,5,1,1,1,2,2,"[(3, 1), (3, 2)]",75,22.666284,NaN,"[(2, 2, 0, 2), (1, 2, 2, 1), (1, 1, 1, 0), (2, 1, 0, 1), (2, 1, 2, 0), (1, 2, 4, 1), (2, 1, 3, 3), (1, 1, 0, 0), (1, 1, 2, 3), (1, 1, 4, 0)]"
3,puzzle_000003,2025-09-12T02:42:33.201175,False,8,0,5,2,1,3,2,1.000000,1,2,0,4,0,2,2,"[(0, 1), (0, 2)]",0,0.004736,NaN,"[(2, 2, 3, 2), (1, 2, 1, 1), (2, 1, 0, 3), (1, 2, 3, 0), (1, 2, 2, 0), (1, 2, 4, 0), (2, 1, 0, 0), (1, 2, 2, 2)]"
4,puzzle_000004,2025-09-12T02:42:33.183368,False,10,4,3,2,1,1,0,2.236068,3,5,3,2,1,1,2,"[(0, 1), (4, 0)]",0,0.010456,NaN,"[(2, 2, 1, 0), (1, 1, 4, 3), (2, 1, 2, 3), (1, 1, 0, 0), (1, 1, 3, 1), (1, 2, 4, 1), (1, 2, 0, 2), (1, 2, 1, 2), (1, 1, 3, 0), (2, 1, 2, 2)]"



 Board Visuals for these samples:

--- Puzzle 1 (Index 0) ---
┌───┬───────┬───┐
│   │       │   │
│   ├───┬───┴───┤
│   │   │       │
├───┼───┤       │
│   │   │       │
│   ├───┼───┬───┤
│   │   │ X │   │
├───┴───┤   ├───┤
│       │ X │   │
└───────┴───┴───┘

--- Puzzle 2 (Index 1) ---
┌───┬───────┬───┐
│   │       │ X │
│   ├───────┼───┤
│   │       │   │
├───┤       │   │
│   │       │   │
├───┼───┬───┼───┤
│   │   │   │   │
├───┤   │   │   │
│ X │   │   │   │
└───┴───┴───┴───┘

--- Puzzle 3 (Index 2) ---
┌───┬───┬───────┐
│   │   │       │
├───┤   │       │
│   │   │       │
├───┼───┴───┬───┤
│   │       │   │
│   ├───────┼───┤
│   │ X   X │   │
├───┼───────┤   │
│   │       │   │
└───┴───────┴───┘

--- Puzzle 4 (Index 3) ---
┌───┬───────┬───┐
│   │ X   X │   │
│   ├───────┤   │
│   │       │   │
├───┴───┬───┴───┤
│       │       │
├───────┼───────┤
│       │       │
├───────┤       │
│       │       │
└───────┴───────┘

--- Puzzle 5 (Index 4) ---
┌───┬───┬───────┐
│   │ X │      

In [188]:
# Dataset Info
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   puzzle_id                   50000 non-null  object 
 1   timestamp                   50000 non-null  object 
 2   is_solvable                 50000 non-null  bool   
 3   total_blocks                50000 non-null  int64  
 4   block_count_1x1             50000 non-null  int64  
 5   block_count_1x2             50000 non-null  int64  
 6   block_count_2x1             50000 non-null  int64  
 7   block_count_2x2             50000 non-null  int64  
 8   goal_initial_row            50000 non-null  int64  
 9   goal_initial_col            50000 non-null  int64  
 10  goal_distance_to_target     50000 non-null  float64
 11  goal_manhattan_distance     50000 non-null  int64  
 12  blocks_between_goal_target  50000 non-null  int64  
 13  adjacent_1x1_count          500

None

In [189]:
# Dataset Duplicate Value Count
print(f'Number of duplicated rows in the dataset: {df.duplicated().sum()}')

Number of duplicated rows in the dataset: 0


In [190]:
# Missing Values/Null Values Count
print(f'There are {df.isna().sum().sum()} missing values in the dataset\n')
df.isna().sum()

There are 50000 missing values in the dataset



puzzle_id                         0
timestamp                         0
is_solvable                       0
total_blocks                      0
block_count_1x1                   0
block_count_1x2                   0
block_count_2x1                   0
block_count_2x2                   0
goal_initial_row                  0
goal_initial_col                  0
goal_distance_to_target           0
goal_manhattan_distance           0
blocks_between_goal_target        0
adjacent_1x1_count                0
adjacent_1x2_count                0
adjacent_2x1_count                0
wall_adjacent_sides               0
empty_spaces_count                0
empty_space                       0
solution_length                   0
solve_time_seconds                0
step_number                   50000
initial_block_states              0
board_visual                      0
dtype: int64

In [191]:
df = df.drop(columns=['step_number','timestamp'])

In [192]:
# Missing Values/Null Values Count again
print(f'There are {df.isna().sum().sum()} missing values in the dataset\n')
df.isna().sum()

There are 0 missing values in the dataset



puzzle_id                     0
is_solvable                   0
total_blocks                  0
block_count_1x1               0
block_count_1x2               0
block_count_2x1               0
block_count_2x2               0
goal_initial_row              0
goal_initial_col              0
goal_distance_to_target       0
goal_manhattan_distance       0
blocks_between_goal_target    0
adjacent_1x1_count            0
adjacent_1x2_count            0
adjacent_2x1_count            0
wall_adjacent_sides           0
empty_spaces_count            0
empty_space                   0
solution_length               0
solve_time_seconds            0
initial_block_states          0
board_visual                  0
dtype: int64

In [193]:
# Quick DataFrame overview (board_visual will show with )
print("\n📊 Basic Statistics:")
display(df.describe())


📊 Basic Statistics:


,total_blocks,block_count_1x1,block_count_1x2,block_count_2x1,block_count_2x2,goal_initial_row,goal_initial_col,goal_distance_to_target,goal_manhattan_distance,blocks_between_goal_target,adjacent_1x1_count,adjacent_1x2_count,adjacent_2x1_count,wall_adjacent_sides,empty_spaces_count,solution_length,solve_time_seconds
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.0,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000
mean,10.036940,3.732840,2.644500,2.659600,1.0,1.365420,0.991380,1.965111,2.361920,3.814620,1.814440,1.589400,1.61864,1.182560,1.658960,13.506300,2.769057
std,1.003731,1.838427,1.218798,1.216107,0.0,1.068808,0.852807,0.824543,1.067388,2.032825,1.402258,1.005976,1.12763,0.716186,0.752186,22.761069,6.002471
min,8.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000968
25%,9.000000,2.000000,2.000000,2.000000,1.0,0.000000,0.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.00000,1.000000,2.000000,0.000000,0.012442
50%,10.000000,4.000000,3.000000,3.000000,1.0,1.000000,1.000000,2.000000,2.000000,4.000000,2.000000,2.000000,1.00000,1.000000,2.000000,0.000000,0.036329
75%,11.000000,4.000000,3.000000,3.000000,1.0,2.000000,2.000000,3.000000,3.000000,5.000000,3.000000,2.000000,2.00000,2.000000,2.000000,24.000000,2.163855
max,15.000000,12.000000,7.000000,6.000000,1.0,3.000000,2.000000,3.162278,4.000000,10.000000,10.000000,6.000000,6.00000,2.000000,2.000000,137.000000,30.047320


In [194]:
def check_min_max_values(df, ignore_columns=None, only_columns=None):
    """
    Check the minimum and maximum values for each variable in the dataframe,
    with the ability to either ignore certain columns or only check a subset.
    
    :param df: The DataFrame
    :param ignore_columns: List of columns to ignore (default is None)
    :param only_columns: List of columns to include (default is None)
    """
    if ignore_columns is None:
        ignore_columns = []  
    if only_columns is None:
        only_columns = df.columns.tolist()  # fallback: use all columns
    
    # Final set of columns to evaluate
    columns_to_check = [col for col in only_columns if col not in ignore_columns]
    
    results = []
    for column in columns_to_check:
        min_value = df[column].min()
        max_value = df[column].max()
        results.append([column, min_value, max_value])
    
    result_df = pd.DataFrame(results, columns=['Column', 'Min Value', 'Max Value'])
    
    print("\nMin and Max Values:")
    display(result_df)

# Example usage
ignore_columns = ['initial_block_states', 'board_visual', 'empty_space']
check_min_max_values(df, ignore_columns)



Min and Max Values:


,Column,Min Value,Max Value
0,puzzle_id,puzzle_000000,puzzle_049999
1,is_solvable,False,True
2,total_blocks,8,15
3,block_count_1x1,0,12
4,block_count_1x2,0,7
5,block_count_2x1,0,6
6,block_count_2x2,1,1
7,goal_initial_row,0,3
8,goal_initial_col,0,2
9,goal_distance_to_target,1.0,3.162278


# Feature Documentation: Klotski Puzzle Dataset

## **Identification & Metadata**
| Feature | Type | Range/Format | Description |
|---------|------|--------------|-------------|
| puzzle_id | Object | "puzzle_XXXXXX" | Unique identifier for each puzzle configuration |
| timestamp | Object | ISO format | When the puzzle was generated or analyzed |
| step_number | Float | 0-200+ | Sequential step number in solution path |

## **Solvability & Solution Metrics**
| Feature | Type | Range/Format | Description |
|---------|------|--------------|-------------|
| is_solvable | Boolean | True/False | Whether the puzzle has a valid solution |
| solution_length | Integer | 0-130+ | Number of moves in optimal solution (0 if unsolvable) |
| solve_time_seconds | Float | 0.0-30.0+ | Computational time taken to solve the puzzle |

## **Block Composition Analysis**
| Feature | Type | Range/Format | Description |
|---------|------|--------------|-------------|
| total_blocks | Integer | 8-15 | Total number of blocks on the board |
| block_count_1x1 | Integer | 0-15 | Number of small 1×1 square blocks |
| block_count_1x2 | Integer | 0-7 | Number of horizontal 1×2 rectangular blocks |
| block_count_2x1 | Integer | 0-6 | Number of vertical 2×1 rectangular blocks |
| block_count_2x2 | Integer | 1 | Number of large 2×2 goal blocks (always 1) |

## **Goal Block Positioning**
| Feature | Type | Range/Format | Description |
|---------|------|--------------|-------------|
| goal_initial_row | Integer | 0-3 | Starting row position of the goal block |
| goal_initial_col | Integer | 0-2 | Starting column position of the goal block |
| goal_distance_to_target | Float | 0.0-4.24 | Euclidean distance from goal to target position |
| goal_manhattan_distance | Integer | 1-4 | Manhattan distance from goal to target position |
| blocks_between_goal_target | Integer | 0-10 | Number of blocks blocking goal's direct path |

## **Spatial Relationship Features**
| Feature | Type | Range/Format | Description |
|---------|------|--------------|-------------|
| adjacent_1x1_count | Integer | 0-4 | Number of 1×1 blocks adjacent to goal block |
| adjacent_1x2_count | Integer | 0-4 | Number of 1×2 blocks adjacent to goal block |
| adjacent_2x1_count | Integer | 0-4 | Number of 2×1 blocks adjacent to goal block |
| wall_adjacent_sides | Integer | 0-4 | Number of board edges touching the goal block |

## **Board Space Analysis**
| Feature | Type | Range/Format | Description |
|---------|------|--------------|-------------|
| empty_spaces_count | Integer | 2-8 | Total number of unoccupied cells on board |
| empty_space | Object | Coordinate pairs | Positions of empty spaces on the board |

## **Visual & State Representation**
| Feature | Type | Range/Format | Description |
|---------|------|--------------|-------------|
| initial_block_states | Object | Shape & Coordinate | Complete state of all block positions |
| board_visual | Object | ASCII art | Visual representation of the puzzle board |


---

## **Feature Engineering Notes**

These features capture multiple dimensions of puzzle complexity:

- **Spatial Complexity**: Block counts, adjacency patterns, empty spaces
- **Goal Accessibility**: Distance metrics, blocking patterns, wall constraints  
- **Solution Difficulty**: Path length, computational complexity, solvability
- **Board Density**: Total blocks vs. available space ratios
- **Strategic Patterns**: Block type distributions and spatial arrangements

This rich feature set enables machine learning models to understand not just the mathematical properties of each puzzle, but also the cognitive and strategic challenges they present to human players.

## **Research Applications**

This dataset supports multiple research directions:

- **Difficulty Prediction**: Predicting human-perceived difficulty from board features
- **Cognitive Load Analysis**: Understanding what makes puzzles mentally challenging
- **Automated Level Design**: Generating puzzles with target difficulty characteristics
- **Player Behavior Modeling**: Analyzing how spatial patterns affect player strategies
- **Educational Applications**: Adaptive difficulty progression in learning systems

In [195]:
import pandas as pd

def check_unique_values(df):
    """
    Check number of unique values for each variable in the dataframe and return a table (DataFrame).
    """
    # Create a list to store results
    results = []
    
    # Iterate over each column to get the number of unique values
    for column in df.columns:
        num_unique = df[column].nunique()  # Number of unique values
        results.append([column, num_unique])
    
    # Create a DataFrame from the results
    result_df = pd.DataFrame(results, columns=['Column', 'Number of Unique Values'])
    
    return result_df

# Example usage
result_df = check_unique_values(df)

# Display the result in table format
display(result_df)  # `display()` is useful in Jupyter Notebooks


,Column,Number of Unique Values
0,puzzle_id,50000
1,is_solvable,2
2,total_blocks,8
3,block_count_1x1,7
4,block_count_1x2,8
5,block_count_2x1,7
6,block_count_2x2,1
7,goal_initial_row,4
8,goal_initial_col,3
9,goal_distance_to_target,6


In [196]:
def analyze_dataset(df):
    """
    Perform analysis on the dataset and display various statistics.
    """
    # Step 1: Calculate Legal Boards (non-zero empty spaces)
    legal_boards = df[df['empty_spaces_count'] > 0]
    num_legal_boards = legal_boards.shape[0]
    
    # Step 2: Solvable and Unsolvable puzzles (based only on legal boards)
    solvable_legal = legal_boards['is_solvable'].sum()
    unsolvable_legal = num_legal_boards - solvable_legal
    
    print(f"Total boards: {df.shape[0]:,}")
    print(f"Legal boards: {num_legal_boards:,}")
    print(f"Solvable puzzles: {solvable_legal:,} ({solvable_legal / num_legal_boards:.1%})")
    print(f"Unsolvable puzzles (based on legal boards): {unsolvable_legal:,} ({unsolvable_legal / num_legal_boards:.1%})")

    # Step 3: Illegal boards (with 0 empty spaces)
    invalid_boards = df[df['empty_spaces_count'] == 0]
    num_invalid_boards = invalid_boards.shape[0]
    percent_invalid_boards = (num_invalid_boards / df.shape[0]) * 100
    print(f"Boards with 0 empty spaces (illegal boards): {num_invalid_boards:,} puzzles ({percent_invalid_boards:.1f}%)")

    # Step 4: Solution length statistics based only on solvable boards
    solvable_df = legal_boards[legal_boards['is_solvable']]  # Use solvable puzzles for solution length analysis
    if len(solvable_df) > 0:
        print(f"Average solution length: {solvable_df['solution_length'].mean():.1f} moves")
        print(f"Solution range: {solvable_df['solution_length'].min()}-{solvable_df['solution_length'].max()} moves")

# Example usage
print("\n")
analyze_dataset(df)




Total boards: 50,000
Legal boards: 41,474
Solvable puzzles: 19,866 (47.9%)
Unsolvable puzzles (based on legal boards): 21,608 (52.1%)
Boards with 0 empty spaces (illegal boards): 8,526 puzzles (17.1%)
Average solution length: 34.0 moves
Solution range: 1-137 moves


In [197]:
df = df[df['empty_spaces_count'] == 2]
analyze_dataset(df)

Total boards: 41,474
Legal boards: 41,474
Solvable puzzles: 19,866 (47.9%)
Unsolvable puzzles (based on legal boards): 21,608 (52.1%)
Boards with 0 empty spaces (illegal boards): 0 puzzles (0.0%)
Average solution length: 34.0 moves
Solution range: 1-137 moves


## 🔧 Advanced Feature Engineering

### 1. Complexity Analyzer Implementation

I'll create a sophisticated complexity scoring system that captures multiple dimensions of puzzle difficulty:

In [198]:
# Feature engineering for structural complexity
def engineer_structural_features(df):
    """
    Engineer structural features following empirical hardness modeling.
    Research-backed feature transformations without learned weights.
    
    This function creates deterministic features that capture puzzle difficulty
    through spatial reasoning, movement constraints, and goal accessibility.
    Each feature is designed based on cognitive psychology and puzzle theory.
    """
    print("🔧 Engineering structural features...")
    
    # ==========================================
    # MOVEMENT CONSTRAINT FEATURES
    # ==========================================
    
    # Mobility Index: Measures how constrained piece movement is
    # Logic: More empty space relative to blocks = easier movement = lower difficulty
    # Formula: empty_spaces / total_blocks
    # Range: ~0.1-0.3 (higher = easier puzzles)
    # Research basis: Spatial reasoning requires maneuvering room (Hegarty & Waller, 2005)
    df['mobility_index'] = (
        df['empty_spaces_count'] / df['total_blocks']
    )
    
    # ==========================================
    # GOAL INTERFERENCE FEATURES  
    # ==========================================
    
    # Blocker Score: Quantifies how much the goal path is obstructed
    # Logic: More blocks blocking + wall constraints = higher clearing complexity
    # Formula: blocks_in_way × (1 + wall_penalty)
    # Wall penalty: Each adjacent wall adds 50% difficulty (max 2 walls for 2x2 goal)
    # Range: 0-30+ (higher = harder puzzles)
    # Research basis: Goal-oriented spatial planning (Knauff & Johnson-Laird, 2002)
    df['blocker_score'] = (
        df['blocks_between_goal_target'] *           # Base obstacle count
        (1 + df['wall_adjacent_sides'] / 2.0)        # Wall constraint multiplier (max 2 walls)
    )
    
    # Clearance Cost: Measures spatial reasoning complexity for goal achievement  
    # Logic: Distance + obstacles = compound planning difficulty
    # Formula: distance_to_goal × blocks_blocking_path
    # Captures both "how far" and "how many obstacles" simultaneously
    # Range: 0-50+ (higher = harder puzzles requiring multi-step planning)
    # Research basis: Working memory load in spatial problem solving (Miyake et al., 2001)
    df['clearance_cost'] = (
        df['goal_distance_to_target'] * df['blocks_between_goal_target']
    )
    
    # ==========================================
    # BOARD COMPOSITION FEATURES
    # ==========================================
    
    # Board Density: Fundamental constraint measure
    # Logic: More filled board = less maneuvering space = higher difficulty
    # Formula: total_blocks / board_size (5×4 = 20 cells)
    # Range: 0.4-0.7 (40%-70% filled)
    # Research basis: Constraint satisfaction problems (Mackworth, 1977)
    df['board_density'] = df['total_blocks'] / 20.0  # Normalize to 5×4 board
    
    # ==========================================
    # FEATURE SELECTION FOR ML PIPELINE
    # ==========================================
    
    # Curated feature set combining:
    # 1. Engineered complexity metrics (mobility, blocker, clearance, density)
    # 2. Raw spatial measurements (distances, adjacencies) 
    # 3. Block composition patterns (different piece types)
    #
    # Selection criteria:
    # - Low correlation (avoid redundancy)
    # - High predictive power for difficulty
    # - Interpretable business meaning
    # - Computationally efficient
    structural_features = [
        # === ENGINEERED COMPLEXITY FEATURES ===
        'mobility_index',          # Movement freedom (lower = harder)
        'blocker_score',          # Goal path obstruction (higher = harder) 
        'clearance_cost',         # Compound spatial reasoning (higher = harder)
        'board_density',          # Space availability (higher = generally harder)
        
        # === RAW SPATIAL MEASUREMENTS ===
        'goal_manhattan_distance', # Direct distance to target position
        'wall_adjacent_sides',    # Wall constraint count (0-2 for Klotski)
        'blocks_between_goal_target', # Direct obstacle count
        
        # === BLOCK ADJACENCY PATTERNS ===
        'adjacent_1x1_count',     # Small block clustering (affects movement)
        'adjacent_1x2_count',     # Horizontal block patterns
        'adjacent_2x1_count'      # Vertical block patterns
    ]
    
    print(f"   ✅ Engineered {len(structural_features)} structural features")
    
    return df, structural_features

# ==========================================
# APPLY FEATURE ENGINEERING PIPELINE
# ==========================================

# Execute the feature engineering process
df, feature_columns = engineer_structural_features(df)

print("\n📊 Structural Feature Summary:")
print("   🔍 Each feature designed to capture different aspects of puzzle difficulty:")
print("   • Lower mobility_index = more constrained movement")
print("   • Higher blocker_score = more goal interference") 
print("   • Higher clearance_cost = more complex spatial reasoning required")
print("   • Higher board_density = less maneuvering space available")

# Display statistical summary of all engineered features
structural_stats = df[feature_columns].describe()
display(structural_stats)

# Add structural features to main feature list for downstream analysis
all_features = feature_columns.copy()


🔧 Engineering structural features...
   ✅ Engineered 10 structural features

📊 Structural Feature Summary:
   🔍 Each feature designed to capture different aspects of puzzle difficulty:
   • Lower mobility_index = more constrained movement
   • Higher blocker_score = more goal interference
   • Higher clearance_cost = more complex spatial reasoning required
   • Higher board_density = less maneuvering space available


,mobility_index,blocker_score,clearance_cost,board_density,goal_manhattan_distance,wall_adjacent_sides,blocks_between_goal_target,adjacent_1x1_count,adjacent_1x2_count,adjacent_2x1_count
count,41474.000000,41474.000000,41474.000000,41474.000000,41474.000000,41474.000000,41474.000000,41474.000000,41474.000000,41474.000000
mean,0.204786,6.133939,8.705244,0.492686,2.359960,1.175242,3.729951,1.812003,1.534817,1.576241
std,0.019264,4.049820,7.173863,0.046845,1.064812,0.717496,1.983253,1.409185,0.987666,1.105195
min,0.142857,0.000000,0.000000,0.400000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.200000,3.000000,2.000000,0.450000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000
50%,0.200000,4.500000,6.000000,0.500000,2.000000,1.000000,4.000000,2.000000,1.000000,1.000000
75%,0.222222,7.500000,13.416408,0.500000,3.000000,2.000000,5.000000,3.000000,2.000000,2.000000
max,0.250000,20.000000,31.622777,0.700000,4.000000,2.000000,10.000000,9.000000,6.000000,6.000000


In [199]:
display(df.head())

,puzzle_id,is_solvable,total_blocks,block_count_1x1,block_count_1x2,block_count_2x1,block_count_2x2,goal_initial_row,goal_initial_col,goal_distance_to_target,goal_manhattan_distance,blocks_between_goal_target,adjacent_1x1_count,adjacent_1x2_count,adjacent_2x1_count,wall_adjacent_sides,empty_spaces_count,empty_space,solution_length,solve_time_seconds,initial_block_states,board_visual,mobility_index,blocker_score,clearance_cost,board_density
0,puzzle_000000,True,11,6,2,2,1,1,2,2.236068,3,6,5,1,0,1,2,"[(3, 2), (4, 2)]",17,2.848788,"[(2, 2, 1, 2), (1, 2, 0, 1), (1, 1, 3, 3), (1, 1, 1, 1), (1, 2, 4, 0), (1, 1, 4, 3), (2, 1, 2, 0), (1, 1, 0, 3), (1, 1, 3, 1), (2, 1, 0, 0), (1, 1, 2, 1)]",┌───┬───────┬───┐\n│ │ │ │\n│ ├───┬───┴───┤\n│ │ │ │\n├───┼───┤ │\n│ │ │ │\n│ ├───┼───┬───┤\n│ │ │ X │ │\n├───┴───┤ ├───┤\n│ │ X │ │\n└───────┴───┴───┘,0.181818,9.0,13.416408,0.55
1,puzzle_000001,False,9,2,1,5,1,1,1,2.000000,2,2,2,1,5,0,2,"[(0, 3), (4, 0)]",0,0.014643,"[(2, 2, 1, 1), (2, 1, 1, 3), (1, 1, 3, 0), (2, 1, 3, 3), (1, 2, 0, 1), (2, 1, 0, 0), (2, 1, 3, 1), (2, 1, 3, 2), (1, 1, 2, 0)]",┌───┬───────┬───┐\n│ │ │ X │\n│ ├───────┼───┤\n│ │ │ │\n├───┤ │ │\n│ │ │ │\n├───┼───┬───┼───┤\n│ │ │ │ │\n├───┤ │ │ │\n│ X │ │ │ │\n└───┴───┴───┴───┘,0.222222,2.0,4.000000,0.45
2,puzzle_000002,True,10,4,2,3,1,0,2,3.162278,4,5,1,1,1,2,2,"[(3, 1), (3, 2)]",75,22.666284,"[(2, 2, 0, 2), (1, 2, 2, 1), (1, 1, 1, 0), (2, 1, 0, 1), (2, 1, 2, 0), (1, 2, 4, 1), (2, 1, 3, 3), (1, 1, 0, 0), (1, 1, 2, 3), (1, 1, 4, 0)]",┌───┬───┬───────┐\n│ │ │ │\n├───┤ │ │\n│ │ │ │\n├───┼───┴───┬───┤\n│ │ │ │\n│ ├───────┼───┤\n│ │ X X │ │\n├───┼───────┤ │\n│ │ │ │\n└───┴───────┴───┘,0.200000,10.0,15.811388,0.50
3,puzzle_000003,False,8,0,5,2,1,3,2,1.000000,1,2,0,4,0,2,2,"[(0, 1), (0, 2)]",0,0.004736,"[(2, 2, 3, 2), (1, 2, 1, 1), (2, 1, 0, 3), (1, 2, 3, 0), (1, 2, 2, 0), (1, 2, 4, 0), (2, 1, 0, 0), (1, 2, 2, 2)]",┌───┬───────┬───┐\n│ │ X X │ │\n│ ├───────┤ │\n│ │ │ │\n├───┴───┬───┴───┤\n│ │ │\n├───────┼───────┤\n│ │ │\n├───────┤ │\n│ │ │\n└───────┴───────┘,0.250000,4.0,2.000000,0.40
4,puzzle_000004,False,10,4,3,2,1,1,0,2.236068,3,5,3,2,1,1,2,"[(0, 1), (4, 0)]",0,0.010456,"[(2, 2, 1, 0), (1, 1, 4, 3), (2, 1, 2, 3), (1, 1, 0, 0), (1, 1, 3, 1), (1, 2, 4, 1), (1, 2, 0, 2), (1, 2, 1, 2), (1, 1, 3, 0), (2, 1, 2, 2)]",┌───┬───┬───────┐\n│ │ X │ │\n├───┴───┼───────┤\n│ │ │\n│ ├───┬───┤\n│ │ │ │\n├───┬───┤ │ │\n│ │ │ │ │\n├───┼───┴───┼───┤\n│ X │ │ │\n└───┴───────┴───┘,0.200000,7.5,11.180340,0.50


In [200]:
new_features = ["mobility_index", "blocker_score", "clearance_cost", "board_density", ]
check_min_max_values(df, only_columns=new_features)


Min and Max Values:


,Column,Min Value,Max Value
0,mobility_index,0.142857,0.250000
1,blocker_score,0.000000,20.000000
2,clearance_cost,0.000000,31.622777
3,board_density,0.400000,0.700000


### 2. Chiral Twin Symmetry Analysis

I'll implement advanced symmetry detection to identify mirror patterns in puzzle boards:

#### Section 1: Core Functions and Data Structures

In [201]:
# Define lightweight block representation for canonical operations
BlockLite = namedtuple('BlockLite', ['num_rows','num_cols','row_pos','col_pos'])

def parse_block_states(s):
    """
    Parse block states with robust error handling
    Converts string representation to list of tuples
    """
    try:
        return ast.literal_eval(s) if isinstance(s, str) else s
    except Exception:
        return []

def blocks_from_states(states):
    """
    Convert state tuples to BlockLite objects
    Creates structured representation for geometric operations
    """
    if not states: 
        return []
    return [BlockLite(h, w, r, c) for (h, w, r, c) in states]

def horizontal_mirror_blocks(blocks, cols=4):
    """
    Create horizontal mirror of blocks (chiral twin transformation)
    
    Logic: Flip each block's column position across vertical centerline
    Formula: new_col = total_cols - original_col - block_width
    """
    return [BlockLite(b.num_rows, b.num_cols, b.row_pos, cols - b.col_pos - b.num_cols) 
            for b in blocks]

def canonical_form(blocks):
    """
    Create ordering-invariant canonical representation
    
    Purpose: Enables comparison regardless of block enumeration order
    Method: Sort blocks by position and size to create unique signature
    """
    return tuple(sorted((b.row_pos, b.col_pos, b.num_rows, b.num_cols) for b in blocks))

print("Chiral twin analysis functions loaded")

Chiral twin analysis functions loaded


#### Section 2: Symmetry Detection & Classification

In [202]:


def is_self_mirror(blocks, cols=4):
    """
    Check if puzzle is its own chiral twin (self-mirroring)
    
    Logic: Compare canonical form with its horizontal mirror
    Returns: True if puzzle is symmetric around vertical centerline
    """
    return canonical_form(blocks) == canonical_form(horizontal_mirror_blocks(blocks, cols))

def mirror_invariant_key(blocks, cols=4):
    """
    Create key identical for a board and its chiral twin
    
    Purpose: Group chiral twins together for efficient pair detection
    Method: Use minimum of canonical form and its mirror (lexicographic ordering)
    Result: Both a puzzle and its chiral twin get the same key
    """
    a = canonical_form(blocks)
    b = canonical_form(horizontal_mirror_blocks(blocks, cols))
    return min(a, b)  # Identical for a board and its chiral twin

print("Symmetry detection functions ready")

Symmetry detection functions ready


#### Section 3: Data Parsing & Canonical Signature Generation

In [203]:


print("Analyzing chiral twin patterns...")
print("="*50)

# Parse block configurations from string format
print("    Parsing puzzle block configurations...")
df['parsed_blocks'] = df['initial_block_states'].apply(parse_block_states)
df['blocks_objects'] = df['parsed_blocks'].apply(blocks_from_states)

print(f"       Parsed {len(df)} puzzle configurations")

# Generate canonical signatures for each puzzle
print("    Computing canonical forms and chiral signatures...")
df['mirror_canonical'] = df['blocks_objects'].apply(canonical_form)
df['mirror_invariant_key'] = df['blocks_objects'].apply(mirror_invariant_key)
df['is_self_mirror'] = df['blocks_objects'].apply(is_self_mirror)

print(f"       Generated canonical signatures for all puzzles")

Analyzing chiral twin patterns...
    Parsing puzzle block configurations...
       Parsed 41474 puzzle configurations
    Computing canonical forms and chiral signatures...
       Generated canonical signatures for all puzzles


#### Section 4: Chiral Twin Grouping & Pair Detection

In [204]:
# Group puzzles by their mirror-invariant signatures
print("    Identifying chiral twin pairs...")
grp = df.groupby('mirror_invariant_key', dropna=False)
df['mirror_group_size'] = grp['puzzle_id'].transform('size')
df['has_chiral_twin'] = (df['mirror_group_size'] >= 2).astype(int)

print(f"       Found {df['has_chiral_twin'].sum():,} puzzles with potential chiral twins")

def pick_twin(group):
    """
    Map each puzzle to its chiral twin within the group
    
    Logic: 
    1. Group puzzles by their canonical form
    2. If 2+ different canonical forms exist, they're chiral twins
    3. Pair puzzles from different canonical forms
    """
    by_canon = {}
    for _, row in group.iterrows():
        by_canon.setdefault(row['mirror_canonical'], []).append(row['puzzle_id'])
    
    twin = {}
    canons = list(by_canon.keys())
    
    # If multiple canonical forms exist, they're chiral twins
    if len(canons) >= 2:
        A, B = by_canon[canons[0]], by_canon[canons[1]]
        # Create reciprocal mapping between twin groups
        for i, a in enumerate(A): 
            twin[a] = B[i % len(B)]
        for j, b in enumerate(B): 
            twin[b] = A[j % len(A)]
    
    # Ensure all puzzles have an entry (None if no twin)
    for ids in by_canon.values():
        for pid in ids: 
            twin.setdefault(pid, None)
    
    return twin

# Create comprehensive twin mapping
print("    Creating chiral twin mappings...")
tmap = {}
for _, g in grp:
    tmap.update(pick_twin(g))

df['chiral_twin_id'] = df['puzzle_id'].map(tmap)

print("    Chiral twin relationships established")

    Identifying chiral twin pairs...
       Found 8,379 puzzles with potential chiral twins
    Creating chiral twin mappings...
    Chiral twin relationships established


#### Section 5: Chiral Twin Analysis Results

In [205]:
print("\n Chiral twin analysis complete!")
print("="*40)

# Summary statistics
twin_count = df['has_chiral_twin'].sum()
self_mirror_count = df['is_self_mirror'].sum()
total_puzzles = len(df)

print(f"    Puzzles with chiral twins: {twin_count:,} ({twin_count/total_puzzles:.1%})")
print(f"    Self-mirroring puzzles: {self_mirror_count:,} ({self_mirror_count/total_puzzles:.1%})")
print(f"    Asymmetric puzzles: {total_puzzles - twin_count - self_mirror_count:,}")

# Add chiral features to main feature list
chiral_features = ['is_self_mirror', 'has_chiral_twin']
all_features.extend(chiral_features)

print(f"\n    Added {len(chiral_features)} chiral symmetry features to analysis")


 Chiral twin analysis complete!
    Puzzles with chiral twins: 8,379 (20.2%)
    Self-mirroring puzzles: 145 (0.3%)
    Asymmetric puzzles: 32,950

    Added 2 chiral symmetry features to analysis


#### Section 6: Chiral Twin Solution Equivalence Testing

In [206]:

print("\n Testing chiral twin solution equivalence...")
print("="*45)

if 'solution_length' in df.columns and 'solve_time_seconds' in df.columns:
    # Find puzzles that have actual chiral twins (not self-mirrors)
    twin_pairs = df[df['chiral_twin_id'].notna() & (df['chiral_twin_id'] != df['puzzle_id'])].copy()
    
    if len(twin_pairs) > 0:
        print(f"    Analyzing {len(twin_pairs)} puzzles with chiral twins")
        
        # Initialize analysis variables
        length_diffs = []
        identical_solutions = 0
        valid_pairs = 0
        
        # Compare each puzzle with its chiral twin
        for _, puzzle in twin_pairs.iterrows():
            twin_id = puzzle['chiral_twin_id']
            twin_row = df[df['puzzle_id'] == twin_id]
            
            if len(twin_row) > 0:
                twin = twin_row.iloc[0]
                
                # Only compare solvable puzzle pairs
                if (puzzle['is_solvable'] and twin['is_solvable'] and 
                    pd.notna(puzzle['solution_length']) and pd.notna(twin['solution_length'])):
                    
                    length_diff = abs(puzzle['solution_length'] - twin['solution_length'])
                    length_diffs.append(length_diff)
                    valid_pairs += 1
                    
                    if length_diff == 0:
                        identical_solutions += 1
        
        # Report quality assessment results
        if length_diffs:
            avg_length_diff = np.mean(length_diffs)
            print(f"\n   SOLUTION EQUIVALENCE RESULTS:")
            print(f"       Average solution length difference: {avg_length_diff:.2f} moves")
            print(f"       Identical solution lengths: {identical_solutions}/{valid_pairs} ({identical_solutions/valid_pairs:.1%})")
            print(f"       Maximum difference observed: {max(length_diffs):.0f} moves")
            
            # Quality interpretation
            if avg_length_diff <= 0.1:
                print(f"\n    EXCELLENT: Chiral twins have nearly identical solution complexity")
                print(f"       This validates our canonical geometry approach")
            elif avg_length_diff <= 1.0:
                print(f"\n    ACCEPTABLE: Minor solution differences between chiral twins")
                print(f"       May indicate solver tie-breaking or minor asymmetries")
            else:
                print(f"\n    INVESTIGATION NEEDED: Significant solution differences")
                print(f"       Average diff {avg_length_diff:.2f} moves suggests systematic issues")
        
        print(f"\n    Valid twin pairs analyzed: {valid_pairs}")
    else:
        print("    No chiral twin pairs found for equivalence testing")
else:
    print("    Missing solution data - skipping equivalence analysis")

print("\n Chiral twin analysis complete!")


 Testing chiral twin solution equivalence...
    Analyzing 4802 puzzles with chiral twins

   SOLUTION EQUIVALENCE RESULTS:
       Average solution length difference: 0.00 moves
       Identical solution lengths: 1063/1063 (100.0%)
       Maximum difference observed: 0 moves

    EXCELLENT: Chiral twins have nearly identical solution complexity
       This validates our canonical geometry approach

    Valid twin pairs analyzed: 1063

 Chiral twin analysis complete!


In [207]:
df.head()

,puzzle_id,is_solvable,total_blocks,block_count_1x1,block_count_1x2,block_count_2x1,block_count_2x2,goal_initial_row,goal_initial_col,goal_distance_to_target,goal_manhattan_distance,blocks_between_goal_target,adjacent_1x1_count,adjacent_1x2_count,adjacent_2x1_count,wall_adjacent_sides,empty_spaces_count,empty_space,solution_length,solve_time_seconds,initial_block_states,board_visual,mobility_index,blocker_score,clearance_cost,board_density,parsed_blocks,blocks_objects,mirror_canonical,mirror_invariant_key,is_self_mirror,mirror_group_size,has_chiral_twin,chiral_twin_id
0,puzzle_000000,True,11,6,2,2,1,1,2,2.236068,3,6,5,1,0,1,2,"[(3, 2), (4, 2)]",17,2.848788,"[(2, 2, 1, 2), (1, 2, 0, 1), (1, 1, 3, 3), (1, 1, 1, 1), (1, 2, 4, 0), (1, 1, 4, 3), (2, 1, 2, 0), (1, 1, 0, 3), (1, 1, 3, 1), (2, 1, 0, 0), (1, 1, 2, 1)]",┌───┬───────┬───┐\n│ │ │ │\n│ ├───┬───┴───┤\n│ │ │ │\n├───┼───┤ │\n│ │ │ │\n│ ├───┼───┬───┤\n│ │ │ X │ │\n├───┴───┤ ├───┤\n│ │ X │ │\n└───────┴───┴───┘,0.181818,9.0,13.416408,0.55,"[(2, 2, 1, 2), (1, 2, 0, 1), (1, 1, 3, 3), (1, 1, 1, 1), (1, 2, 4, 0), (1, 1, 4, 3), (2, 1, 2, 0), (1, 1, 0, 3), (1, 1, 3, 1), (2, 1, 0, 0), (1, 1, 2, 1)]","[(2, 2, 1, 2), (1, 2, 0, 1), (1, 1, 3, 3), (1, 1, 1, 1), (1, 2, 4, 0), (1, 1, 4, 3), (2, 1, 2, 0), (1, 1, 0, 3), (1, 1, 3, 1), (2, 1, 0, 0), (1, 1, 2, 1)]","((0, 0, 2, 1), (0, 1, 1, 2), (0, 3, 1, 1), (1, 1, 1, 1), (1, 2, 2, 2), (2, 0, 2, 1), (2, 1, 1, 1), (3, 1, 1, 1), (3, 3, 1, 1), (4, 0, 1, 2), (4, 3, 1, 1))","((0, 0, 1, 1), (0, 1, 1, 2), (0, 3, 2, 1), (1, 0, 2, 2), (1, 2, 1, 1), (2, 2, 1, 1), (2, 3, 2, 1), (3, 0, 1, 1), (3, 2, 1, 1), (4, 0, 1, 1), (4, 2, 1, 2))",False,1,0,None
1,puzzle_000001,False,9,2,1,5,1,1,1,2.000000,2,2,2,1,5,0,2,"[(0, 3), (4, 0)]",0,0.014643,"[(2, 2, 1, 1), (2, 1, 1, 3), (1, 1, 3, 0), (2, 1, 3, 3), (1, 2, 0, 1), (2, 1, 0, 0), (2, 1, 3, 1), (2, 1, 3, 2), (1, 1, 2, 0)]",┌───┬───────┬───┐\n│ │ │ X │\n│ ├───────┼───┤\n│ │ │ │\n├───┤ │ │\n│ │ │ │\n├───┼───┬───┼───┤\n│ │ │ │ │\n├───┤ │ │ │\n│ X │ │ │ │\n└───┴───┴───┴───┘,0.222222,2.0,4.000000,0.45,"[(2, 2, 1, 1), (2, 1, 1, 3), (1, 1, 3, 0), (2, 1, 3, 3), (1, 2, 0, 1), (2, 1, 0, 0), (2, 1, 3, 1), (2, 1, 3, 2), (1, 1, 2, 0)]","[(2, 2, 1, 1), (2, 1, 1, 3), (1, 1, 3, 0), (2, 1, 3, 3), (1, 2, 0, 1), (2, 1, 0, 0), (2, 1, 3, 1), (2, 1, 3, 2), (1, 1, 2, 0)]","((0, 0, 2, 1), (0, 1, 1, 2), (1, 1, 2, 2), (1, 3, 2, 1), (2, 0, 1, 1), (3, 0, 1, 1), (3, 1, 2, 1), (3, 2, 2, 1), (3, 3, 2, 1))","((0, 0, 2, 1), (0, 1, 1, 2), (1, 1, 2, 2), (1, 3, 2, 1), (2, 0, 1, 1), (3, 0, 1, 1), (3, 1, 2, 1), (3, 2, 2, 1), (3, 3, 2, 1))",False,1,0,None
2,puzzle_000002,True,10,4,2,3,1,0,2,3.162278,4,5,1,1,1,2,2,"[(3, 1), (3, 2)]",75,22.666284,"[(2, 2, 0, 2), (1, 2, 2, 1), (1, 1, 1, 0), (2, 1, 0, 1), (2, 1, 2, 0), (1, 2, 4, 1), (2, 1, 3, 3), (1, 1, 0, 0), (1, 1, 2, 3), (1, 1, 4, 0)]",┌───┬───┬───────┐\n│ │ │ │\n├───┤ │ │\n│ │ │ │\n├───┼───┴───┬───┤\n│ │ │ │\n│ ├───────┼───┤\n│ │ X X │ │\n├───┼───────┤ │\n│ │ │ │\n└───┴───────┴───┘,0.200000,10.0,15.811388,0.50,"[(2, 2, 0, 2), (1, 2, 2, 1), (1, 1, 1, 0), (2, 1, 0, 1), (2, 1, 2, 0), (1, 2, 4, 1), (2, 1, 3, 3), (1, 1, 0, 0), (1, 1, 2, 3), (1, 1, 4, 0)]","[(2, 2, 0, 2), (1, 2, 2, 1), (1, 1, 1, 0), (2, 1, 0, 1), (2, 1, 2, 0), (1, 2, 4, 1), (2, 1, 3, 3), (1, 1, 0, 0), (1, 1, 2, 3), (1, 1, 4, 0)]","((0, 0, 1, 1), (0, 1, 2, 1), (0, 2, 2, 2), (1, 0, 1, 1), (2, 0, 2, 1), (2, 1, 1, 2), (2, 3, 1, 1), (3, 3, 2, 1), (4, 0, 1, 1), (4, 1, 1, 2))","((0, 0, 1, 1), (0, 1, 2, 1), (0, 2, 2, 2), (1, 0, 1, 1), (2, 0, 2, 1), (2, 1, 1, 2), (2, 3, 1, 1), (3, 3, 2, 1), (4, 0, 1, 1), (4, 1, 1, 2))",False,1,0,None
3,puzzle_000003,False,8,0,5,2,1,3,2,1.000000,1,2,0,4,0,2,2,"[(0, 1), (0, 2)]",0,0.004736,"[(2, 2, 3, 2), (1, 2, 1, 1), (2, 1, 0, 3), (1, 2, 3, 0), (1, 2, 2, 0), (1, 2, 4, 0), (2, 1, 0, 0), (1, 2, 2, 2)]",┌───┬───────┬───┐\n│ │ X X │ │\n│ ├───────┤ │\n│ │ │ │\n├───┴───┬───┴───┤\n│ │ │\n├───────┼───────┤\n│ │ │\n├───────┤ │\n│ │ │\n└───────┴───────┘,0.250000,4.0,2.000000,0.40,"[(2, 2, 3, 2), (1, 2, 1, 1), (2, 1, 0, 3), (1, 2, 3